In [1]:
import os
import itertools

prop_dict = {
    0: 'parp1_qed_sa',
    1: 'fa7_qed_sa',
    2: '5ht1b_qed_sa',
    3: 'braf_qed_sa',
    4: 'jak2_qed_sa',
}

guidance_weight_dict = {
    0: 0.5,
    1: 0.7,
    2: 0.3,
}

weight_decays = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.5]

hyper_id_list = [
    {
        "wd": wd[0]
    }
    for wd in itertools.product(
        weight_decays,
    )
]

print(f"Number of hyperparameter combinations: {len(hyper_id_list) - 1}")

Number of hyperparameter combinations: 8


In [2]:
prop_train_configs = [f for f in os.listdir(os.getcwd()) if f.endswith('.yaml')]
for f in prop_train_configs:
    os.remove(f)

In [3]:
for i in range(5):
    for j, hypers in enumerate(hyper_id_list):
        config = f"""data: 
    data: ZINC250k
    dir: './data'
    batch_size: 1024
    max_node_num: 38
    max_feat_num: 9

sde:
    x:
        type: VP
        beta_min: 0.1
        beta_max: 1.0
        num_scales: 1000
    adj:
        type: VE
        beta_min: 0.2
        beta_max: 1.0
        num_scales: 1000

model:
    model: Regressor
    depth: 3
    nhid: 16
    dropout: 0

train:
    prop: {prop_dict[i]}
    num_epochs: 10
    lr: 0.001
    lr_schedule: False
    reg_type: weight_decay
    weight_decay: {hypers['wd']:1.1e}
    lr_decay: 0.999
    eps: 1.0e-5"""

        with open(f'prop_train_{i}_{j}.yaml', 'w') as f:
            f.write(config)
    
    for k in range(len(guidance_weight_dict)):
        sample_config=f"""data:
  data: ZINC250k
  dir: './data'

model:
  diff:
    ckpt: gdss_zinc250k_v2
    predictor: Reverse
    corrector: Langevin
    snr: 0.2
    scale_eps: 0.8
    n_steps: 1
  prop:
    ckpt: weight_decay/prop_{prop_dict[i]}
    weight_x: {guidance_weight_dict[k]}   # 0.5, 0.4, 0.6, 0.7, and 0.6 for parp1, fa7, 5ht1b, braf, and jak2, respectively
    weight_adj: 0

sample:
  noise_removal: True
  probability_flow: False
  eps: 1.0e-3
  num_samples: 3000
  ood: 0.04
        """

        with open(f'sample_{i}_{k}.yaml', 'w') as f:
            f.write(sample_config)